In [ ]:
import dlib
import cv2
import os
import re
import json
from pylab import *
from PIL import Image, ImageChops, ImageEnhance

In [ ]:
os.makedirs('dataset/real', exist_ok=True)
os.makedirs('dataset/fake', exist_ok=True)

In [ ]:
#processing videos to extract facial regions and save them as images in a dataset folder
#used Dlib for face detection and OpenCV for video processing and image saving
train_frame_folder = 'train_sample_videos'
with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
    data = json.load(file)
list_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]
detector = dlib.get_frontal_face_detector()

'''
It iterates through each video file in the list, opens the video using OpenCV, and then processes each frame.
Inside the loop, it calculates the frame rate and iterates through the frames at a regular interval.
For each selected frame, it uses the Dlib face detector to detect faces in the frame.
For each detected face, it extracts the face region, resizes it to 128x128 pixels, and saves it as a PNG file in either the 'dataset/real' or 'dataset/fake' folder based on the label from the metadata.'''

for vid in list_of_train_data:
    count = 0
    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))
    frameRate = cap.get(5)
    while cap.isOpened():
        frameId = cap.get(1)            #Retrieves the current frame index
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % ((int(frameRate)+1)*1) == 0:               # way of selecting frames at regular intervals
            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                crop_img = frame[y1:y2, x1:x2]
                if data[vid]['label'] == 'REAL':
                    cv2.imwrite('dataset/real/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                elif data[vid]['label'] == 'FAKE':
                    cv2.imwrite('dataset/fake/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                count+=1